In [3]:
# Tensor Init

import torch
torch.manual_seed(0)

# list -> tensor
arr = [1,2]
tensor = torch.tensor(arr)

# float -> tensor
val = 2.0
tensor = torch.tensor(val)

# numpy -> tensor 
import numpy as np
np_arr = np.array([1,2])
x_t = torch.from_numpy(np_arr)

# 2x3 tensor of zeros
zeros_t = torch.zeros((2,3))
# 2x3 tensor of ones
ones_t = torch.ones((2,3)) 
# 2x3 tensor of random numbers
rand_t = torch.randn((2,3))

In [5]:
# Tensor Attributes

# torch size
zeros_t.shape

# torch type
x_t = torch.tensor(2.0)
x_t.dtype

# 해당 텐서의 할당 장치 확인
arr = [1,2]
x_t = torch.tensor(arr, dtype=torch.float32)
x_t.device

# tesnsor 속성 수정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
arr = [1,2]
x_t = torch.tensor(arr, dtype=torch.float32, device=device)
x_t = x_t.to(device, dtype=torch.int)

In [ ]:
# Tensor Operations

# scalar mul
c = 10
x_t = x_t*c

# sum, element wise-sum (sub 동일)
x1_t = torch.zeros((1,2))
x2_t = torch.ones((1,2))
x1_t + x2_t

# mul
x1_t = torch.tensor([[1,2],[3,4]])
x2_t = torch.tensor([[1,2,3],[4,5,6]])
torch.matmul(x1_t, x2_t)

In [ ]:
# Gradients(f의 편미분값 계산)

x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)
z = torch.tensor(1.5, requires_grad=True)
f = x**2+y**2+z**2
f.backward()
x.grad, y.grad, z.grad


In [ ]:
# nn module -> 모델 정의, 훈련, 테스트 관련 기본 기능 제공

import torch.nn as nn


in_dim, out_dim = 256, 10 # 입력 차원, 출력 차원 설정
vec = torch.randn(256)
layer = nn.Linear(in_dim, out_dim, bias=True) # 단일 레이어 정의
out = layer(vec)

W = torch.rand(10,256) # 행렬 W
b = torch.zeros(10,1) # 편향 벡터 b
out = torch.matmul(W, vec) + b

in_dim, feature_dim, out_dim = 784, 256, 10
vec = torch.randn(784)
layer1 = nn.Linear(in_dim, feature_dim, bias=True)
layer2 = nn.Linear(feature_dim, out_dim, bias=True)
out = layer2(layer1(vec))

relu = nn.ReLU() # 비선형 함수 추가를 ReLU 활용
out = layer2(relu(layer1(vec)))
     

In [ ]:
class BaseClassifier(nn.Module):
  def __init__(self, in_dim, feature_dim, out_dim):
    super(BaseClassifier, self).__init__()
    self.layer1 = nn.Linear(in_dim, feature_dim, bias=True)
    self.layer2 = nn.Linear(feature_dim, out_dim, bias=True)
    self.relu = nn.ReLU()

  # 분류기
  def forward(self, x):
    x = self.layer1(x)
    x = self.relu(x)
    out = self.layer2(x)
    return out
     

In [ ]:
# optimizer: 기울기 기반 파라미터 조정

from torch import optim

lr = 1e-3
optimizer = optim.SGD(classifier.parameters(), lr=lr)

optimizer.step() # Updates parameters via SGD
optimizer.zero_grad() # Zeroes out gradients between minibatches

In [ ]:
import os
from PIL import Image
from torchvision import transforms

# Dataset Load
class ImageDataset(Dataset):
  def __init__(self, img_dir, label_file):
    super(ImageDataset, self).__init__()
    self.img_dir = img_dir
    self.labels = torch.tensor(np.load(label_file, allow_pickle=True))
    self.transforms = transforms.ToTensor()

  # index를 입력으로 받아 해당 index의 예시를 반환
  def __getitem__(self, idx):
    img_pth = os.path.join(self.img_dir, "img_{}.jpg".format(idx))
    img = Image.open(img_pth)
    img = self.transforms(img).flatten()
    label = self.labels[idx]
    return {"data":img, "label":label}
      
  # dataset의 길이, 모델을 훈련 or 테스트할 예시 데이터의 수를 의미
  def __len__(self):
    return len(self.labels)

In [ ]:
train_dataset = ImageDataset(img_dir='./data/train/',
                             label_file='./data/train/labels.npy')

# DataLoader: 데이터셋 인스턴스화를 입력으로 사용
# 미니배치에 의해 데이터셋에 로드하고 epoch 간에 dataset을 shuffle
# Python의 다중 처리 내장 모듈을 사용하여 미니배치를 병렬로 효율적으로 사용
train_loader = DataLoader(train_dataset, 
                          batch_size=4, 
                          shuffle=True)

In [ ]:
# MNIST 분류기 생성

import matplotlib.pyplot as plt
import torch
from torch import optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

torch.manual_seed(0)

class BaseClassifier(nn.Module):
  def __init__(self, in_dim, feature_dim, out_dim):
    super(BaseClassifier, self).__init__()
    self.classifier = nn.Sequential(
        nn.Linear(in_dim, feature_dim, bias=True),
        nn.ReLU(),
        nn.Linear(feature_dim, out_dim, bias=True)
    )
    
  def forward(self, x):
    return self.classifier(x)
    

# MNIST 데이터셋을 로드한다.
train_dataset = MNIST(".", train=True, 
                      download=True, transform=ToTensor())
test_dataset = MNIST(".", train=False, 
                     download=True, transform=ToTensor())
train_loader = DataLoader(train_dataset, 
                          batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, 
                         batch_size=64, shuffle=False)

# model, optimizer, hyperparameter를 인스턴스화한다.
in_dim, feature_dim, out_dim = 784, 256, 10
lr=1e-3
loss_fn = nn.CrossEntropyLoss()
epochs=40
classifier = BaseClassifier(in_dim, feature_dim, out_dim)
optimizer = optim.SGD(classifier.parameters(), lr=lr)

def train(classifier=classifier,
          optimizer=optimizer,
          epochs=epochs,
          loss_fn=loss_fn):

  classifier.train()
  loss_lt = []
  for epoch in range(epochs):
    running_loss = 0.0
    for minibatch in train_loader:
      data, target = minibatch
      data = data.flatten(start_dim=1)
      out = classifier(data)
      computed_loss = loss_fn(out, target)
      computed_loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      # 각 미니배치 손실 합 기록
      running_loss += computed_loss.item()
    loss_lt.append(running_loss/len(train_loader))
    print("Epoch: {} train loss: {}".format(epoch+1, running_loss/len(train_loader)))

  plt.plot([i for i in range(1,epochs+1)], loss_lt)
  plt.xlabel("Epoch")
  plt.ylabel("Training Loss")
  plt.title(
      "MNIST Training Loss: optimizer {}, lr {}".format("SGD", lr))
  plt.show()

  # checkpoint state 저장
  torch.save(classifier.state_dict(), 'mnist.pt')

def test(classifier=classifier, 
          loss_fn = loss_fn):
  classifier.eval()
  accuracy = 0.0
  computed_loss = 0.0

  with torch.no_grad():
      for data, target in test_loader:
          data = data.flatten(start_dim=1)
          out = classifier(data)
          _, preds = out.max(dim=1)

          # loss and accuracy 계산
          computed_loss += loss_fn(out, target)
          accuracy += torch.sum(preds==target)
          
      print("Test loss: {}, test accuracy: {}".format(
          computed_loss.item()/(len(test_loader)*64), accuracy*100.0/(len(test_loader)*64)))

In [ ]:
# CIFAR-10을 위한 합성공 신경망 구축

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
        )
        self.block2 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(9216, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128,10),
            nn.BatchNorm1d(10)
        )

    def forward(self, x):
        x = self.block1(x)
        return self.block2(x)

In [ ]:
# ResNet34 아키텍쳐 구현

from torchvision.models import resnet34

model = resnet34()

# 2재의 합성공 layer로 구성
class ResidualBlock(nn.Module):
  def __init__(self, in_layers, out_layers, downsample=None):
    super(ResidualBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_layers, out_layers,
                           kernel_size=3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(out_layers)
    self.conv2 = nn.Conv2d(out_layers, out_layers,
                           kernel_size=3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(out_layers)
    self.downsample = downsample
    self.relu = nn.ReLU(inplace=True)

  def forward(self, inp):
    # Residual block
    out = self.conv1(inp)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    
    if self.downsample: 
      inp = self.downsample(inp)
    
    # 단축 연결
    out += inp
    return out

In [ ]:
class ResNet34(nn.Module):
  def __init__(self):
    super(ResNet34, self).__init__()

    self.conv1 = nn.Sequential(
      nn.Conv2d(3, 64, kernel_size=7,
                stride=2, padding=3, bias=False),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3,
                   stride=2, padding=1)
    )

    # 각 Residual Block은 convolution layer를 2개씩 가진다.
    # 연속  3개의 block, 6개의 convolution layer
    self.comp1 = nn.Sequential(
      ResidualBlock(64, 64),
      ResidualBlock(64, 64),
      ResidualBlock(64, 64)
    )

     # 연속 4개의 block, 8개의 convolution layer
    downsample1 = nn.Sequential(
      nn.Conv2d(64, 128, kernel_size=1,
             stride=1, bias=False),
      nn.BatchNorm2d(128)
    )
    self.comp2 = nn.Sequential(
      ResidualBlock(64, 128, downsample=downsample1),
      ResidualBlock(128, 128),
      ResidualBlock(128, 128),
      ResidualBlock(128, 128)
    )
    
    # 연속 6개의 block, 12개의 convolution layer
    downsample2 = nn.Sequential(
      nn.Conv2d(128, 256, kernel_size=1, stride=1, bias=False),
      nn.BatchNorm2d(256)
    )
    self.comp3 = nn.Sequential(
      ResidualBlock(128, 256, downsample=downsample2),
      ResidualBlock(256, 256),
      ResidualBlock(256, 256),
      ResidualBlock(256, 256),
      ResidualBlock(256, 256),
      ResidualBlock(256, 256),
    )
    
    # 연속 3개의 block, 6개의 convolution layer
    downsample3 = nn.Sequential(
      nn.Conv2d(256, 512, kernel_size=1, stride=1, bias=False),
      nn.BatchNorm2d(512)
    )
    self.comp4 = nn.Sequential(
      ResidualBlock(256, 512, downsample=downsample3),
      ResidualBlock(512, 512),
      ResidualBlock(512, 512)   
    )

    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    # ImageNet classifier: 1000 classes
    self.fc = nn.Linear(512, 1000)

  def forward(self, inp):
    out = self.conv1(inp)
    
    out = self.comp1(out)
    out = self.comp2(out)
    out = self.comp3(out)
    out = self.comp4(out)

    out = self.avgpool(out)
    out = torch.flatten(out, 1)
    out = self.fc(out)

    return out